In [ ]:
#import gymnasium as gym
import gym #for using stable baselines comparison
import os
import numpy as np
np.random.seed(42)

In [ ]:
env = gym.make("MountainCar-v0")
#env=gym.make("CartPole-v1")
#env = gym.make("MountainCar-v0" , render_mode="human")
#env.reset()
env.close()
# done = False
# while not done:
#     action = 2  # always go right!
#     env.step(action)
#     env.render()

In [ ]:
env.observation_space
#a)first array is min value for position and velocity
#b)second array is max value for position and velocity

In [ ]:
for i in range(10):
    res=env.action_space.sample()
    print(f"{i}-th sampling gave {res} as the action")
    

In [ ]:
print(env.observation_space.high) #max-min position of distance and max-min value of velocity
print(env.observation_space.low)

In [ ]:
dis = 20 #poso tha "temaxistei" to observation kai action space
DISCRETE_OS_SIZE = [dis, dis]
discrete_os_win_size = (env.observation_space.high - env.observation_space.low)/DISCRETE_OS_SIZE
print(discrete_os_win_size)

In [ ]:
0.09*20 #is equal to [-1.2,0.6]

# Q-Table


In [ ]:
# Q-Learning settings
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 4*25001
EPISODES

In [ ]:
#greedy epsilon
# Exploration settings
epsilon = 1  # not a constant, qoing to be decayed
#high epsilon -> agent prefers exploration
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES//2 #apo ayto to epeisodio kai meta to e = 0 = constant
epsilon_decay_value = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)
END_EPSILON_DECAYING

In [ ]:
EPISODES//2

In [ ]:
# epsilon_old = []
# epsilon=1
# for ep in range(EPISODES):
#     epsilon_old=np.append(epsilon_old, epsilon)
#     if END_EPSILON_DECAYING >= ep >= START_EPSILON_DECAYING:
            
#             epsilon -= epsilon_decay_value
# len(epsilon_old[epsilon_old < 0])  
# #len(epsilon_old)
# # #output = 0 

In [ ]:
# epsilon_old

In [ ]:
# len(np.unique(epsilon_old)) == len(epsilon_old) # some episodes have same epsilon
# #output = False

In [ ]:
(DISCRETE_OS_SIZE + [env.action_space.n])
#20 possible x positions
#20 possible velocity values
#3 possible actions

In [ ]:
def get_discrete_state(state):
    if type(state) == tuple: #if epeidh to env.reset() epistrefei tuple, enw to env.step()[0] epistrefei array
        #to new_state
        state = state[0]
    discrete_state = (state - env.observation_space.low)/discrete_os_win_size
    #return tuple(discrete_state.astype(int)) #older version?
    return tuple(discrete_state.astype(np.int64))

In [ ]:
same_env_reset=env.reset()
print(f"state discrete: {get_discrete_state(same_env_reset)} \nenv output :{same_env_reset}")

In [ ]:
q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))
q_table.shape

In [ ]:
random_state=env.reset()
q_table[get_discrete_state(random_state)] #epistrefei to Q-value gia ta 3 actions
#o agent epilegei to megisto Q-value
print(f"q-table={q_table[get_discrete_state(random_state)]} \nargmax = {np.argmax(q_table[get_discrete_state(random_state)])}")

In [ ]:
q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))
for ep in range(EPISODES):
    current_state = get_discrete_state(env.reset())
    i=0 #agent's moves counter
    done=False #it becomes true from env.step()
    while not done:
        #choose best action for current_state
        action=np.argmax(q_table[current_state])
        #after action, we have a new state and the reward for the action that moved us "forward"
        new_state, reward, done, truncated = env.step(action)
        new_state_x=new_state[0] #first value is x position #check x position for env.goal_position
        new_state=get_discrete_state(new_state) #discretize our new state
        #Qmax: apo ton mh ananewmeno pinaka, vriskw to megisto Q gia to 
        Q_max = np.max(q_table[new_state])
        #Q: apo thn mhn ananewmeno pinaka, to Q to gia state kai to action poy phra
        Q_current=q_table[current_state + (action,)]
        #eq
        new_q=(1 - LEARNING_RATE) * Q_current + LEARNING_RATE * (reward + DISCOUNT * Q_max)
        #update Q table
        q_table[current_state + (action,)] = new_q        
        if new_state_x >= env.goal_position:
            q_table[current_state + (action,)]=0
            #print(f"megalo X")
        if done==True and ep%5000==0:
            print(f"Done at {i}-th move durin {ep}-th episode.")
        else:
            i=i+1
        current_state=new_state

#output
# Done at 199-th move durin 0-th episode.
# Done at 173-th move durin 5000-th episode.
# Done at 170-th move durin 10000-th episode.
# Done at 174-th move durin 15000-th episode.
# Done at 110-th move durin 20000-th episode.
# Done at 110-th move durin 25000-th episode.
# Done at 110-th move durin 30000-th episode.
# Done at 110-th move durin 35000-th episode.
# Done at 172-th move durin 40000-th episode.
# Done at 110-th move durin 45000-th episode.
# Done at 110-th move durin 50000-th episode.
# Done at 111-th move durin 55000-th episode.
# Done at 172-th move durin 60000-th episode.
# Done at 108-th move durin 65000-th episode.
# Done at 108-th move durin 70000-th episode.
# Done at 108-th move durin 75000-th episode.
# Done at 115-th move durin 80000-th episode.
# Done at 107-th move durin 85000-th episode.
# Done at 107-th move durin 90000-th episode.
# Done at 108-th move durin 95000-th episode.
# Done at 109-th move durin 100000-th episode.

# Save trained q_table locally

In [ ]:
np.save(os.path.join('trained_q_table'), q_table)


# Load trained q_table locally

In [ ]:
q_table=np.load(os.path.join('trained_q_table.npy'))

In [ ]:
len(q_table[q_table>0])

In [ ]:
# q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))
# epsilon=1
# for ep in range(EPISODES):
#     current_state = get_discrete_state(env.reset())
#     i=0 #agent's moves counter
#     done=False #it becomes true from env.step()
#     while not done:
#         if np.random.random() > epsilon:
#         #choose best action for current_state
#             action=np.argmax(q_table[current_state])
#         else:
#             action=np.random.randint(0, env.action_space.n)
#         #after action, we have a new state and the reward for the action that moved us "forward"
#         new_state, reward, done, truncated , information = env.step(action)
#         new_state_x=new_state[0] #first value is x position #check x position for env.goal_position
#         new_state=get_discrete_state(new_state) #discretize our new state
#         #Qmax: apo ton mh ananewmeno pinaka, vriskw to megisto Q gia to 
#         Q_max = np.max(q_table[new_state])
#         #Q: apo thn mhn ananewmeno pinaka, to Q to gia state kai to action poy phra
#         Q_current=q_table[current_state + (action,)]
#         #eq
#         new_q=(1 - LEARNING_RATE) * Q_current + LEARNING_RATE * (reward + DISCOUNT * Q_max)
#         #update Q table
#         q_table[current_state + (action,)] = new_q        
#         if new_state_x >= env.goal_position:
#             q_table[current_state + (action,)]=0
#             #print(f"megalo X")
#         if done==True and ep%5000==0:
#             print(f"Done with {i}-th move during {ep}-th episode. Epsilon={epsilon}")
#         else:
#             i=i+1
#         current_state=new_state
#     if END_EPSILON_DECAYING >= ep >= START_EPSILON_DECAYING:
#         epsilon -= epsilon_decay_value
#         #print(f"epsilon value = {epsilon}")
    
# #output


In [ ]:
# goal_position=env.goal_position
# max_speed=env.observation_space.high[1]
# min_speed=env.observation_space.low[1]
# velocity=np.random.uniform(min_speed,max_speed)
# goal_state=(goal_position,velocity)
# get_discrete_state(goal_state)

In [ ]:
# max_position=env.observation_space.high[0]
# min_position=env.observation_space.low[0]
# print(f"max position:{max_position}, min position:{min_position}")

In [ ]:
# get_discrete_state((max_position, velocity))

In [ ]:
# max_speed=env.observation_space.high[1]
# min_speed=env.observation_space.low[1]
# np.random.uniform(min_speed,max_speed)

# Manual training VS Stable Baselines

In [ ]:
    current_state = get_discrete_state(env.reset())
    i=0 #agent's moves counter
    done=False #it becomes true from env.step()
    while not done:
        #choose best action for current_state
        action=np.argmax(q_table[current_state])
        #after action, we have a new state and the reward for the action that moved us "forward"
        new_state, reward, done, truncated = env.step(action)
        #new_state_x=new_state[0] #first value is x position #check x position for env.goal_position
        new_state=get_discrete_state(new_state) #discretize our new state
#         #Qmax: apo ton mh ananewmeno pinaka, vriskw to megisto Q gia to 
#         Q_max = np.max(q_table[new_state])
#         #Q: apo thn mhn ananewmeno pinaka, to Q to gia state kai to action poy phra
#         Q_current=q_table[current_state + (action,)]
#         #eq
#         new_q=(1 - LEARNING_RATE) * Q_current + LEARNING_RATE * (reward + DISCOUNT * Q_max)
#         #update Q table
#         q_table[current_state + (action,)] = new_q        
#         if new_state_x >= env.goal_position:
#             q_table[current_state + (action,)]=0
#             #print(f"megalo X")
#         if done==True and ep%5000==0:
#             print(f"Done at {i}-th move durin {ep}-th episode.")
#         else:
        i=i+1
        current_state=new_state
    print(f"Number of steps until Done: {i+1}")

In [ ]:
from stable_baselines3 import DQN

dqn_model = DQN("MlpPolicy",
            env,
            verbose=1,
            train_freq=16,
            gradient_steps=8,
            gamma=0.99,
            exploration_fraction=0.2,
            exploration_final_eps=0.07,
            target_update_interval=600,
            learning_starts=1000,
            buffer_size=10000,
            batch_size=128,
            learning_rate=4e-3,
            policy_kwargs=dict(net_arch=[256, 256]),
            #tensorboard_log=tensorboard_log,
            seed=2)
     
dqn_model.learn(total_timesteps=10000, log_interval=4)


In [ ]:
obs=env.reset()
done=False
i=0
while not done:
    
    action, wut = dqn_model.predict(obs)
    new_state, reward, done, truncated = env.step(action)
    obs=new_state
    i=i+1
print(f"Number of steps until Done: {i+1}")